# Load Shedding, Shifting and Elastic Demand

This example is a simple illustration that aims to explain the use of load shedding, load shifting and elastic demand in PyPSA. The example is designed to simulate a single day of electrical consumption in a small network with solar generation, which could represent a single house or a small group of houses.

In [ ]:
import pypsa
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd

### Create network

In [ ]:
network = pypsa.Network()

### Build the snapshots we consider for the first T hours in 2023


In [ ]:
network.set_snapshots(pd.date_range("2023-01-01 04:00", "2023-01-01 20:00", freq="H"))

print(network.snapshots)

### Add fuel types

In [ ]:
network.add("Carrier", "solar")
network.add("Carrier", "load")
network.add("Carrier", "battery")

### Add buses

In [ ]:
network.add("Bus", "My bus")

### Add generators - Load shedding

Solar PV panel generation per unit of capacity is considered only in the daytime.

"load shedding" is introduced as a generator to illustrate the decrease in demand, thereby enhancing the generation. It is essential when the network's generation is insufficient to fulfill the required demand

In [ ]:
# Add solar generation
pv_pu = [
    0.0,
    0.0,
    0.0,
    0.2,
    0.4,
    0.65,
    0.85,
    0.9,
    0.85,
    0.65,
    0.4,
    0.3,
    0.2,
    0.1,
    0.0,
    0.0,
    0.0,
]
network.add(
    "Generator",
    "solar",
    bus="My bus",
    p_nom=0.4,
    carrier="solar",
    p_max_pu=pv_pu,
    marginal_cost=0 + 0.01 * np.random.random(),
)

# Add load shedding as generator
network.add(
    "Generator",
    "load shedding",
    bus="My bus",
    p_nom=50,
    carrier="load",
    p_max_pu=1.0,
    marginal_cost=1e2,
    p_nom_extendable=True,
)

### Add constant base load values

In [ ]:
load = pd.Series(0.2, index=range(17))  # constant baseload
network.add("Load", "baseload", bus="My bus", p_set=load.values)

### Add storage units to the network - Load shifting

Storage units could be used to represent load shifting in PyPSA, where energy use is in peak hours is shifted to off-peak hours, which is a similar behaviour to discharging a storage unit in peak hours and charging it in off-peak hours.

Smart thermostats can shift the timing of energy use by automatically reducing heating or cooling during peak demand periods and increasing it during off-peak periods.

In [ ]:
network.add(
    "StorageUnit",
    "smart thermostat",
    bus="My bus",
    p_nom=0,
    carrier="battery",
    marginal_cost=4 * np.random.random(),
    p_nom_extendable=True,
    p_max_pu=1,
    p_min_pu=-1,
    efficiency_store=0.9,
    efficiency_dispatch=0.95,
    standing_loss=0.01,
    cyclic_state_of_charge=True,
    max_hours=6,
)

### Optimize the network and plot some results

In [ ]:
network.lopf()

In [ ]:
fig, ax = plt.subplots()

network.generators_t.p.plot(ax=ax)
network.storage_units_t.p.plot(ax=ax)
network.loads_t.p.plot(ax=ax)

ax.set_ylabel("Power (MW)")

We can observe from the least cost solution the following:
- The smart thermostat is increasing the demand during the peak solar generation, to save energy when there is a lack of supply
- In the first and last hours of the day, load shedding is applied since there is not enough storage or generation available

## Elastic Demand

Elastic demand is demand that is changing with price changes. The demand response to price changes is usually nonlinear and sometimes random in the case of a human action to high or low prices. In this example, the demand was assumed as a linear response price changes, for simplicity.

### Creating networks

Several networks will be created in a number of iterations that represent adjusted demand after price changes. The networks use the same inputs as the previous one except for the demand value. So the first iteration will use the original demand value that was in the previous network. Then get the price of the network from the formula in the script. Finally the adjusted demand value is calculated from the assumed linear relation, and used in the next iteration.

In [ ]:
base_load = 0.2
all_networks = []

for _ in range(6):
    network = pypsa.Network()
    network.set_snapshots(pd.date_range("2023-01-01 04:00", "2023-01-01 20:00", freq="H"))
    network.add("Carrier", "solar")
    network.add("Carrier", "load")
    network.add("Carrier", "battery")
    network.add("Bus", "My bus")
    pv_pu = [0.0, 0.0, 0.0, 0.2, 0.4, 0.65, 0.85, 0.9, 0.85, 0.65, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0]
    network.add("Generator", "solar", bus="My bus", p_nom=0.4, carrier="solar", p_max_pu=pv_pu,
                marginal_cost=0 + 0.01 * np.random.random())
    network.add("Generator", "load shedding", bus="My bus", p_nom=50, carrier="load", p_max_pu=1.0, marginal_cost=1e2,
                p_nom_extendable=True)
    load = pd.Series(base_load, index=range(17))  # variable baseload based on base_load_values
    network.add("Load", "baseload", bus="My bus", p_set=load.values)
    network.add("StorageUnit", "smart thermostat", bus="My bus", p_nom=0, carrier="battery",
                marginal_cost=4 * np.random.random(), p_nom_extendable=True, p_max_pu=1, p_min_pu=-1,
                efficiency_store=0.9, efficiency_dispatch=0.95, standing_loss=0.01, cyclic_state_of_charge=True,
                max_hours=6)
    network.lopf()

    # Estimated price of the network in euros per MWh
    network_cost_per_MWh = (network.objective + network.objective_constant) / network.loads_t.p.sum().sum()
    
    # assuming a linear demand response to price
    new_load = (network_cost_per_MWh - 120) / -300
    base_load = new_load
    
    all_networks.append(network)


### Demand and price values from the created networks

In [ ]:
average_loads = []
prices = []
for n in all_networks:
    average_loads.append(n.loads_t.p.mean().sum())
    prices.append((n.objective + n.objective_constant) / n.loads_t.p.sum().sum())

### Plotting demand against price

In [ ]:
first_demand_iteration = average_loads[0]
updated_average_loads = average_loads[1:] # demand value in first iteration didn't have a price
updated_prices = prices[:-1] # last price value wasn't used

# Scatter plot of the points
plt.scatter(updated_average_loads, updated_prices, label='Elastic demand') 

# Add numbers to each point
for i, (x, y) in enumerate(zip(updated_average_loads, updated_prices)):
    plt.annotate('Iteration '+str(i + 1), (x, y), textcoords="offset points", xytext=(30, 5), ha='center')

# Fit a linear curve to the data
slope, intercept = np.polyfit(updated_average_loads, updated_prices, 1)

# Linear curve
x = np.linspace(min(updated_average_loads), max(updated_average_loads), 100)
y = slope * x + intercept
plt.plot(x, y, label=f'Linear curve', color='blue')

plt.xlabel('Demand (MW)')
plt.ylabel('Price (Euro/MWh)')
plt.title('Elasticity Curve')
plt.legend(loc='upper right')
plt.xlim([0.15, 0.35])
plt.ylim([35, 60])
plt.plot(x, y)
plt.show()

We can observe from the elastic demand the following:
- The initial base demand of 0.2 resulted in a price that leads the system to increase demand to around 0.27, meaning such price is considered a bit low.
- The next price resulting from the 0.27 demand is considered high. Therefore, the next iteration would lower the demand, thus lowering the price.
- This keeps going until the demand converges to a value around 0.24, which is the equilibrium of the assumed linear relationship.